In [22]:
# import required libraries
import pandas as pd
import string
from nltk.corpus import stopwords

In [23]:
#Get the spam data collection 
spam_data = pd.read_table(r'C:\Users\user\Desktop\Vinni study folder\my python notebooks\Practice datasets\SpamCollection',names = ['Response','messages'])
# OR pd.read_csv('path',sep='\t') .......... to tell that the data is tab seperated and not comma seperated

In [24]:
spam_data.head()

,Response,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [25]:
# View more information using describe method
spam_data.describe()

,Response,messages
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [26]:
#view response 
spam_data['Response'].value_counts()

ham     4825
spam     747
Name: Response, dtype: int64

In [27]:
# or
spam_data.groupby('Response').describe()

messages                                                            \
            count unique                                                top   
Response                                                                      
ham          4825   4516                             Sorry, I'll call later   
spam          747    653  Please call our customer service representativ...   

               
         freq  
Response       
ham        30  
spam        4

In [28]:
#Verify length of the messages and also add it as a new column 
length = []
for i in range (spam_data.shape[0]):
    len_of_msg = len(spam_data.iloc[i,1])
    length.append(len_of_msg)
    

In [29]:
spam_data['Message length'] = length

In [30]:
spam_data.head()

,Response,messages,Message length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [34]:
#define a function to get rid of stopwords present in the messages
def remove_stopwords(mess) :
    no_punctuation = [ char for char in mess if char not in string.punctuation ]
    no_punctuation = ''.join(no_punctuation)
    a = [char for char in no_punctuation.split() if char.lower() not in stopwords.words('english')]
    return a
    
    

In [35]:
spam_data['messages'].apply(remove_stopwords)

0       [Go, jurong, point, crazy, Available, bugis, n...
1                          [Ok, lar, Joking, wif, u, oni]
2       [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3           [U, dun, say, early, hor, U, c, already, say]
4       [Nah, dont, think, goes, usf, lives, around, t...
                              ...                        
5567    [2nd, time, tried, 2, contact, u, U, £750, Pou...
5568                   [ü, b, going, esplanade, fr, home]
5569                     [Pity, mood, Soany, suggestions]
5570    [guy, bitching, acted, like, id, interested, b...
5571                                   [Rofl, true, name]
Name: messages, Length: 5572, dtype: object

In [36]:
spam_data.head()

,Response,messages,Message length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [37]:
#start text processing with vectorizer 
from sklearn.feature_extraction.text import CountVectorizer

In [38]:
#use bag of words by applying the function and fit the data into it
bag_of_words = CountVectorizer(analyzer = remove_stopwords).fit(spam_data['messages'])

In [39]:
#print length of bag of words stored in the vocabulary_ attribute
print (len(bag_of_words.vocabulary_))

11425


In [41]:
# storing bag of words using transform method
message_bagofwords = bag_of_words.transform(spam_data['messages'])

In [46]:
#apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(message_bagofwords)

In [52]:
#print shape of the tfidf 
tfidf = tfidf_transformer.transform(message_bagofwords)
print (tfidf.shape)

(5572, 11425)


In [56]:
#choose naive Bayes model to detect the spam and fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(tfidf,spam_data['Response'])

In [62]:
#check model for the predicted and expected value say for message#2 and message#5
message = spam_data['messages'][10]
bag_word = bag_of_words.transform([message])
tfidf_message = tfidf_transformer.transform(bag_word)

print('predicted:',spam_detect_model.predict(tfidf_message)[0])
print('expected:',spam_data['Response'][10])

predicted: ham
expected: ham
